In [3]:
%gui qt5
# need to wait for gui qt5 to completely load before running next cell
import numpy as np
from neurom.io import swc
import napari

Read swc downloaded from http://www.neuromorpho.org/neuron_info.jsp?neuron_name=204-2-6nj

data_block is np.array-like with the following columns:
    
    [X, Y, Z, R, TYPE, ID, P]
    X(float): x-coordinate
    Y(float): y-coordinate
    Z(float): z-coordinate
    R(float): radius
    TYPE(integer): one of the types described by POINT_TYPE
    ID(integer): unique integer given to each point, the `ROOT_ID` is -1
    P(integer): the ID of the parent

In [4]:
data = swc.read('204-2-6nj.CNG.swc').data_block
# data2 = data[data[:,6].argsort()]

In [5]:
# function to convert data into list of paths to be passed into napari
def parse_swc(data):
    
    # compute location of branches
    break_points = [0] + list(np.nonzero(np.diff(data[:, 6]) < 0)[0]+1) + [len(data)-1]
    paths = []
    
    # use location of branches to separate branches into individual paths
    for i in range(len(break_points)-1):
        if break_points[i+1] - break_points[i] > 2:
            paths.append(data[break_points[i]:break_points[i+1], :3])
    return paths

In [6]:
paths = parse_swc(data)
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(paths, shape_type='path', edge_color='blue', name='swc')

<Shapes layer 'swc' at 0x12b04e8d0>

![Screenshot from napari. napari does not have ability to retreive snapshot from 3D render](napari_neurons.png)

Edge widths from data can vary from each edge. napari cannot scale edge widths

In [26]:
# radii from swc file
swc_radii = data[:,3]

# radii from napari paths
napari_radii = viewer.layers['swc'].edge_width

print("Edge widths from swc (first 5)")
print(swc_radii[0:5],'\n')
print("Edge width from napari path (one radius for all edges)")
print(napari_radii)

Edge widths from swc (first 5)
[5.2517 5.2517 5.2517 0.425  0.425 ] 

Edge width from napari path (one radius for all edges)
1
